In [12]:
import requests
import OSMPythonTools as osm

In [52]:
from OSMPythonTools.api import Api
api = Api()
way = api.query('way/5887599')
way

In [22]:
from OSMPythonTools.api import Api
api = Api()
way = api.query('way/5887599')


print(way.tag('building'))
print(way.tag('architect'))
print(way.tag('website'))

castle
Johann Lucas von Hildebrandt
https://www.belvedere.at/


In [227]:
from OSMPythonTools.overpass import Overpass

overpass = Overpass()
result = overpass.query('nwr["addr:city"="Kraków"]["tourism"]; out;')

s = set({})
for dupa in result.elements():
  # node: dupa.lat(), dupa.lon()
  print(dupa)

In [67]:
from OSMPythonTools.overpass import Overpass, overpassQueryBuilder

overpass = Overpass()
result = overpassQueryBuilder()

0

In [311]:
from OSMPythonTools.nominatim import Nominatim
import json

def user_search(x:float, y:float) -> str:
    # 1° of latitude = always 111.32 km
    eps = 0.0005
    x0, x1 = x - eps, x + eps
    y0, y1 = y - eps, y + eps

    overpass = Overpass()
    result = overpass.query(f'nwr["tourism"]({x0},{y0},{x1},{y1}); out;')

    s = set({})
    for obj in result.elements():
        name = obj.tag("name")
        description = gen_description(obj)
        address = gen_address(obj)
        category = obj.tag("tourism")
        latitide, longitude = get_lat_lon(obj)
        
        picture_url = ""

        if None in (name, address, category, latitide, longitude, picture_url):
            continue
        s.add((name, description, address, category.capitalize(),
              latitide, longitude, picture_url))
    return s

user_search(50.0641425, 19.9231397, "sklep")

{('Muzeum Geologiczne WGGiOŚ AGH',
  '',
  '30-059 Kraków, Aleja Adama Mickiewicza 30',
  'Museum',
  50.0641425,
  19.9231397,
  '')}

## Fajne funkcje

In [338]:
def gen_description(obj):
  txt = ""
  if not (website := obj.tag("website")) is None:
    txt += "Witryna internetowa: " + website + "\n"
  if not (hours := obj.tag("opening_hours")) is None:
    txt += "Godziny otwarcia: " + hours + "\n"
  if not (wheelchair := obj.tag("wheelchair")) is None:
    txt += "Przystosowanie dla osób niepełnosprawnych: " + wheelchair + "\n"
  if not (fee := obj.tag("fee")) is None:
    txt += "Opłaty: " + fee + "\n"
  if not (phone := obj.tag("phone")) is None:
    txt += "Telefon: " + phone + "\n"
  if not (address := gen_address(obj)) is None:
    txt += "Adres: " + address  + "\n"
    
  return txt

def gen_address(obj):
  r = [ obj.tag("addr:postcode"), " ", obj.tag("addr:city"), ", ", obj.tag("addr:street"), " ", obj.tag("addr:housenumber") ]
  for i in r:
    if i is None:
      return None
  return "".join(r)
  
def get_lat_lon(obj):
  if not obj.lat() is None and not obj.lon() is None:
    return obj.lat(), obj.lon()
  elif len(obj.nodes()) == 0:
    return  None, None
  elif not obj.nodes()[0].lat() is None and not obj.nodes()[0].lon() is None:
    return obj.nodes()[0].lat(), obj.nodes()[0].lon()
  else:
    return None, None

def search_for_cool_objects(city):
    # cools found in Krakow: hotel hostel information motel gallery camp_site theme_park apartment zoo attraction guest_house museum
    COOLS = {"information", "gallery", "camp_site",
             "theme_park", "zoo", "attraction", "museum"}

    overpass = Overpass()
    result = overpass.query(f'nwr["addr:city"="{city}"]["tourism"]; out;')

    s = set({})
    for obj in result.elements():
        if not obj.tag("tourism") in COOLS:
            continue

        name = obj.tag("name")
        description = gen_description(obj)
        address = gen_address(obj)
        category = obj.tag("tourism")
        latitide, longitude = get_lat_lon(obj)
        
        picture_url = ""

        if None in (name, address, category, latitide, longitude, picture_url):
            continue
        s.add((name, description, address, category.capitalize(),
              latitide, longitude, picture_url))
    return s


search_for_cool_objects("Kraków")

TypeError: can only concatenate str (not "NoneType") to str

## Photos